In this notebook we're going to take a close look at the model. We'll start off by defining some constant that we'll use throughout the notebook.

In [ ]:
CHECKPOINT_PATH = "./checkpoints/checkpoint.pt"

Then we'll load the checkpoint and instantiate the UltraZoom upscaler.

In [ ]:
import torch

from src.ultrazoom.model import UltraZoom

checkpoint = torch.load(
    CHECKPOINT_PATH,
    map_location="cpu",
    weights_only=True,
)

model = UltraZoom(**checkpoint["upscaler_args"])

model.add_weight_norms()

state_dict = checkpoint["upscaler"]

# Compensate for compiled state dict.
for key in list(state_dict.keys()):
    state_dict[key.replace("_orig_mod.", "")] = state_dict.pop(key)

model.load_state_dict(state_dict)

model.remove_parameterizations()

print("Model loaded successfully")

The first module I want to analyze is the Adaptive Residual Mix. I want to see if the model is utilizing its newfound mixing abilities. Let's start by looking at the values of the `alpha` scalar to see if the modules are learning to mix the input and residuals or if it's opting for the identity function.

In [ ]:
from src.ultrazoom.model import AdaptiveResidualMix

print("Adaptive Residual Mix alphas:")

for i, module in enumerate(model.modules()):
    if isinstance(module, AdaptiveResidualMix):
        alpha = torch.sigmoid(module.alpha).item()

        l2_norm = module.conv.weight.norm(p=2.0, dim=1).mean().item()

        print(f"Module {i}: alpha = {alpha:.4f}, weight norm = {l2_norm:.4f}")